[Data Visualization](https://infovis.fh-potsdam.de/tutorials/) · FH Potsdam · Summer 2023

# Tutorial 3: Data wrangling

In this tutorial you will learn a few tricks for getting data into shape for visualization. You will see how data can be loaded, parsed, and examined. For this we will continue to work with the **Pandas** package, in particular with the DataFrame data structure, and get to know a few additional helpers. 

In [1]:
import pandas as pd

## Loading 

The first step is to bring the data into the purview of your notebook. So regardless of data structure and format, you need to have access to the dataset. We will briefly cover four common ways of loading data into your Jupyter notebook: direct entry, local file, URL and API.

### Enter data directly

The simplest way to add data to your notebook is to enter it verbatim into the notebook as we have seen with the capital cities in the first tutorial:

In [2]:
cities = pd.DataFrame({
  "name": ["Athens", "Bratislava", "Copenhagen", "Dublin"],
  "area": [39, 367.6, 86.2, 115],
  "elevation": [170, 152, 14, 20],
  "population": [664046, 429564, 602481, 553165]
  }
)

cities

,name,area,elevation,population
0,Athens,39.0,170,664046
1,Bratislava,367.6,152,429564
2,Copenhagen,86.2,14,602481
3,Dublin,115.0,20,553165


✏️ *Add a column for years when you have visited or plan to visit these cities*

### Open a local file

You might also want to open a local file. This notebook comes with an example file `anscombe.json` that you can browse on the right side in the FILES section. 

We can open a file using Python's built-in `open()` method, after which we can `read()` its contents into the variable `anscombe_json` and finally `close()` it again. In this case the data is in the JSON format, which we will need to parse. We'll get to this later. You can open all kinds of formats. Here we know that we are dealing with a JSON file because of its extension.


In [3]:
file = open("anscombe.json")
anscombe_json = file.read()
file.close()

print(anscombe_json)

[
  {"Series":"I", "X":10.0, "Y":8.04},
  {"Series":"I", "X":8.0, "Y":6.95},
  {"Series":"I", "X":13.0, "Y":7.58},
  {"Series":"I", "X":9.0, "Y":8.81},
  {"Series":"I", "X":11.0, "Y":8.33},
  {"Series":"I", "X":14.0, "Y":9.96},
  {"Series":"I", "X":6.0, "Y":7.24},
  {"Series":"I", "X":4.0, "Y":4.26},
  {"Series":"I", "X":12.0, "Y":10.84},
  {"Series":"I", "X":7.0, "Y":4.81},
  {"Series":"I", "X":5.0, "Y":5.68},

  {"Series":"II", "X":10.0, "Y":9.14},
  {"Series":"II", "X":8.0, "Y":8.14},
  {"Series":"II", "X":13.0, "Y":8.74},
  {"Series":"II", "X":9.0, "Y":8.77},
  {"Series":"II", "X":11.0, "Y":9.26},
  {"Series":"II", "X":14.0, "Y":8.10},
  {"Series":"II", "X":6.0, "Y":6.13},
  {"Series":"II", "X":4.0, "Y":3.10},
  {"Series":"II", "X":12.0, "Y":9.13},
  {"Series":"II", "X":7.0, "Y":7.26},
  {"Series":"II", "X":5.0, "Y":4.74},

  {"Series":"III", "X":10.0, "Y":7.46},
  {"Series":"III", "X":8.0, "Y":6.77},
  {"Series":"III", "X":13.0, "Y":12.74},
  {"Series":"III", "X":9.0, "Y":7.11},
 

✏️ *Take a look in the `data` directory under Files in the right sidebar and try to open any other file*

### Get data via a URL

There are some methods that can directly load a dataset via a URL, i.e., a web address. For others you might have to retrieve the file first to continue parsing it. The `requests` package helps you to send HTTP requests and retrieve the responses. 

In the following, the news feed of Tagesschau is retrieved via an HTTP GET request. Note that the news feed is made available as an XML format; of course you can retrieve all kinds of file formats using this method:

In [4]:
import requests

response = requests.get('https://www.tagesschau.de/xml/rss2/')
tagesschau_xml = response.text
print(tagesschau_xml[:1000]) # this displays the first 1000 characters

<?xml version='1.0' encoding='UTF-8'?>
<rss version="2.0" xmlns:content="http://purl.org/rss/1.0/modules/content/" xmlns:dc="http://purl.org/dc/elements/1.1/">
  <channel>
    <title>tagesschau.de - Die Nachrichten der ARD</title>
    <link>https://www.tagesschau.de/infoservices/alle-meldungen-100.html</link>
    <description>Die aktuellen Beiträge der Seite https://www.tagesschau.de/infoservices/alle-meldungen-100.html</description>
    <language>de</language>
    <copyright>ARD-aktuell / tagesschau.de</copyright>
    <lastBuildDate>Thu, 01 Jun 2023 19:30:37 +0200</lastBuildDate>
    <pubDate>Thu, 01 Jun 2023 19:30:37 +0200</pubDate>
    <docs>http://blogs.law.harvard.edu/tech/rss</docs>
    <ttl>90</ttl>
    <dc:publisher>tagesschau.de</dc:publisher>
    <dc:language>de</dc:language>
    <dc:rights>ARD-aktuell / tagesschau.de</dc:rights>
    <dc:date>2023-04-14T07:19:20Z</dc:date>
    <dc:source>tagesschau.de</dc:source>
    <item>
      <title>Liveblog: ++ Niederlande wollen "Leopar

✏️ *Find the news feed for another webpage and try to load it*

### Use an API

Some web platforms require the use of an API (application programming interface) to get access to their data. Simply put, an API is a structured way to request and retrieve data. Oftentimes it is just a specific way to format the URL.

The German National Library offers an [API to query Entity Facts](https://www.dnb.de/EN/Professionell/Metadatendienste/Datenbezug/Entity-Facts/entityFacts_node.html) contained in the GND (Gemeinsame Normdatei). In this case the API provides the data in the JSON format, which has become quite common for web APIs, but you will also encounter many other formats. 

To retrieve information for a given GND entity by its id, such as the GND entry for the artist [Käthe Kollwitz](https://en.wikipedia.org/wiki/Käthe_Kollwitz) you have to put the `base_url` together with the `gnd_id`: 

In [5]:
base_url = "https://hub.culturegraph.org/entityfacts/"
gnd_id = "118564943" # GND identifier found at the Wikipedia page on Käthe Kollwitz
gnd_response = requests.get(base_url+gnd_id).text

print(gnd_response[:1000])

{
  "@context" : "https://hub.culturegraph.org/entityfacts/context/v1/entityfacts.jsonld",
  "@id" : "https://d-nb.info/gnd/118564943",
  "describedBy" : {
    "valid" : "2023-06-01T17:31:45Z",
    "license" : "https://creativecommons.org/publicdomain/zero/1.0/",
    "@id" : "https://hub.culturegraph.org/entityfacts/118564943"
  },
  "preferredName" : "Käthe Kollwitz",
  "surname" : "Kollwitz",
  "forename" : "Käthe",
  "@type" : "person",
  "variantName" : [ "Kėte Kolʹvic", "K. Kolʹvic", "Kathe Kollwitz", "Kaethe Kollwitz", "Kaisui Kelehuizhi", "Kaisui Kelehuizhi", "Kelehuizhi", "Kėte Kollʹvic", "Käte Kollwitz", "Кете Кольвиц", "Кете Шмидт", "Ke te Kol bi cheu", "Ke te Kol bi cheu", "K'et'e K'olbich'ŭ", "K'et'e K'olbich'ŭ", "케테 콜비츠", "콜비츠 케테", "凯绥 珂勒惠之", "珂勒惠之", "凯绥 珂勒惠之", "ケーテ コルヴィッツ" ],
  "realIdentity" : [ {
    "preferredName" : "Käthe Schmidt"
  } ],
  "dateOfBirth" : "8. Juli 1867",
  "dateOfDeath" : "22. April 1945",
  "placeOfBirth" : [ {
    "@id" : "http

✏️ *Prepare an API request with a GND id of another person of German history*

## Parsing 

Apart from our little cities example, so far we have only loaded the data into unstructured strings. To be able to analyze the data, we have to turn the unstructured strings of symbols into a practical data structure of the DataFrame that we can work with. This process is commonly referred to as ‘parsing’. 

As we have seen above, data can come in various file formats, which are in turn more or less appropriate for particular data structures. We'll cover four typical ones in the following section, but we will see more over the course of the tutorials to come.

The different ways of loading data (e.g., by file path or URL) are independent from the particular data formats provided. For example, you can load CSV data from a local file or from a web address. While the files typically indicate with the extension what format they have, URLs or APIs may not have these. If it is not clear, you may have to check the documentation or take a peek into the file.

### CSV

The CSV format is probably the most common file format in the context of data analysis and visualization. CSV files contain tabular data that can be viewed and edited in a spreadsheet software such as Excel. CSV stands for [comma-separated values](https://en.wikipedia.org/wiki/Comma-separated_values), which seems to say it all: the data values are separated by commas and each row represents one item. However, there are also CSV files that use separators other than commas, such as tabs and semicolons. 

Let's load a CSV file! Thankfully Pandas has the convenient `read_csv()` method ready for us, which can open CSV data via a file path or URL, and turns it directly into a DataFrame object. 

Here we're retrieving information about [childcare places in Potsdam](https://opendata.potsdam.de/explore/dataset/kitaboerse-20161108/export/) from Potsdam's Open Data portal. This file happens to be in a CSV format that uses the semicolon as a separator (hence `sep=";"` as the second parameter in the method call):

In [6]:
kitas = pd.read_csv(
  "https://opendata.potsdam.de/explore/dataset/kitaboerse-20161108/download/",
  sep=";")

kitas.head()

,name_der_kindertagesbetreuungseinrichtung,stand_vom,barrierefreie_einrichtung,kinderkrippe_0_3_j,tagespflege_0_3_j,padagogisch_begleitete_spielgruppe_0_3_j,kindergarten_3_j_schuleintritt,hort_ab_schuleintritt,andere_kinderbetreuung_ab_3_klasse,strasse,...,abweichende_offnungszeiten,ubernachtung_moglich,schliesstage_von_bis,fruhstuck,mittag,vesper,abendessen,versorgungsart,link_zum_bereich_kinder_und_jugend_der_landeshauptstadt_potsdam,link_zu_anmeldeinformationen_der_einrichtung
0,"Wurzelwerk, AWO, Kita",15.08.2016,Nein,Ja,Nein,Nein,Ja,Nein,Nein,Zum Kahleberg,...,NaN,Nein,NaN,Ja,Ja,Ja,Nein,Mischversorgung,http://www.potsdam.de/kita-tipp,http://www.awo-potsdam.de/awo-bezirksverband/a...
1,"Baumhaus, Kita",07.11.2016,Ja,Nein,Nein,Nein,Ja,Ja,Nein,Sternstr.,...,NaN,Nein,keine,Ja,Ja,Ja,Nein,Mischversorgung,http://www.potsdam.de/kita-tipp,http://www.baumhaus-potsdam.de
2,"Sonnenland, Hort",21.09.2016,Nein,Nein,Nein,Nein,Nein,Ja,Nein,Knobelsdorffstr.,...,Ferien 6 bis 17 Uhr,Nein,NaN,Nein,Ja,Ja,Nein,Eigenversorgung am Standort,http://www.potsdam.de/kita-tipp,https://www.ejf.de/kindertagesstaetten-horte/h...
3,"Pfiffikus, AWO, Kita",08.09.2016,Ja,Ja,Nein,Nein,Ja,Ja,Nein,Pietschkerstr.,...,NaN,Nein,NaN,Ja,Ja,Ja,Nein,Eigenversorgung geliefert,http://www.potsdam.de/kita-tipp,http://www.awo-potsdam.de/einrichtungen-und-di...
4,Evangelische Kindertagesstätte Sonnenblume,01.02.2016,Nein,Ja,Nein,Nein,Ja,Nein,Nein,Bellavitestr.,...,Fr 6 bis 17 Uhr,Nein,1.8.2016-12.8.2016,Ja,Ja,Ja,Nein,Eigenversorgung am Standort,http://www.potsdam.de/kita-tipp,http://www.hoffbauer-bildung.de/kita-sonnenblu...


✏️ *Try loading another CSV dataset from [Potsdam's Open Data Portal](https://opendata.potsdam.de/)*

### JSON

[JSON](https://en.wikipedia.org/wiki/JSON) is a format that grew in popularity in the context of web development. It stands for JavaScript Object Notation and makes for a compact, yet relatively human-readable format.

Above we have already opened the contents of the local file `anscombe.json` using the `file.read()` method. To parse the contents into the structured form of a DataFrame, with which we can do some analysis, we can rely on Pandas and the handy `read_json()` method. Let's do this with the anscombe data we already opened:

In [7]:
anscombe = pd.read_json(anscombe_json)
anscombe

,Series,X,Y
0,I,10,8.04
1,I,8,6.95
2,I,13,7.58
3,I,9,8.81
4,I,11,8.33
5,I,14,9.96
6,I,6,7.24
7,I,4,4.26
8,I,12,10.84
9,I,7,4.81


### XML

[XML](https://en.wikipedia.org/wiki/XML) (Extensible Markup Language) is a data format, which can have very different kinds of hierarchical structures. XML files are common in a wide variety of contexts, including libraries, and especially in situations, in which the interoperability of multiple systems by several vendors needs to be ensured.

The 🌲 **ElementTree** module will help us to parse the elements contained in an XML file. 

In [8]:
import xml.etree.ElementTree as ET

As we have already retrieved the XML feed from Tagesschau (and saved it in the variable `tagesschau_xml`), we can now parse it directly from the string, i.e., using the method `ET.fromstring()`:

In [9]:
tagesschau = ET.fromstring(tagesschau_xml)
tagesschau

<Element 'rss' at 0x13ed06b10>

This gives the root element of the XML feed (and all its children) in the variable `tagesschau`. 

Going through all items with `findall` and searching within these for specific sub-elements with `find`, we can extract the publication date and time and the title of the respective item. For this we can use the [XPath](https://en.wikipedia.org/wiki/XPath) syntax. In the following example, these elements are put together into the DataFrame `tagesschau_df`. Note that it helps to peek into the XML source of the feed to know the specific element names.

In [10]:
# create two empty lists
dates = []
titles = []

# go through all item elements in the tree
for item in tagesschau.findall('.//item'):
  # extract date information and titles
  dates.append( item.find('.//pubDate').text )
  titles.append( item.find('.//title').text )

# create a dataframe containing the two columns
tagesschau_df = pd.DataFrame(    
    {'date': dates,
     'title': titles
    })  

tagesschau_df

,date,title
0,"Thu, 01 Jun 2023 19:30:37 +0200","Liveblog: ++ Niederlande wollen ""Leopard""-Panz..."
1,"Thu, 01 Jun 2023 19:21:33 +0200","NATO, Schweden und die Ukraine: Näher dran, ab..."
2,"Thu, 01 Jun 2023 19:17:13 +0200",Ungarn-Resolution des EU-Parlaments: Mehr als ...
3,"Thu, 01 Jun 2023 19:00:53 +0200",Welche Firmen besonders vom KI-Boom profitieren
4,"Thu, 01 Jun 2023 18:24:20 +0200",Marktbericht: DAX erholt sich zum Monatsbeginn
5,"Thu, 01 Jun 2023 18:00:17 +0200",DeutschlandTrend: Zufriedenheit mit Ampel-Regi...
6,"Thu, 01 Jun 2023 17:59:26 +0200",Anschlag auf Pipelines: Durchsuchung in Deutsc...
7,"Thu, 01 Jun 2023 17:34:53 +0200",EU-Parlament äußert Bedenken über ungarischen ...
8,"Thu, 01 Jun 2023 17:22:43 +0200","Gesetzliche Krankenversicherung: Milliarden, d..."
9,"Thu, 01 Jun 2023 17:15:37 +0200",Finanzminister Lindner macht Ministerien konkr...


✏️ *Each news item also contains a `description` element. Why not add a third column to the DataFrame?*

### HTML

While not really a data format per se, many datasets are actually published via the document format of the web: [HTML](https://en.wikipedia.org/wiki/HTML) (hypertext markup language). All webpages are in one way or another provided in the form of HTML.

Turning HTML content into structured data, also known as scraping, is necessary when information is published on webpages, while not (yet) available as structured data sets. For this scenario, the module 🍜 **Beautiful Soup** is your friend! To use it you can import the latest version by referring to `bs4`:


In [11]:
import bs4

Suppose you want to extract the table of cities on the Wikipedia page [List of largest cities](https://en.wikipedia.org/wiki/List_of_largest_cities) you would first get the content of the page and then involve the magic powers of Beautiful Soup, some of which might look a bit cryptic at this point:


In [12]:
permalink = "https://en.wikipedia.org/w/index.php?title=List_of_largest_cities&oldid=952676625"
wiki = requests.get(permalink).text

In the variable `wiki` we now have the HTML content of the Wikipedia page. Next we initialize Beautiful Soup's `html.parser` and engage in a bit of data parsing:

In [13]:
soup = bs4.BeautifulSoup(wiki, "html.parser")

# extract all table elements
tables = soup.find_all('table')

# create some empty lists of city names and populations
names = []
pops = []

for table in tables:
  # iterate over all rows,  <tr> is the html element for table rows
  rows = table.find_all('tr')    
  for row in rows:
    # get all cells per row,  <td> is the html element for table cells
    cells = row.find_all('td')
    if len(cells) > 1:
      # get span elements from cells in population column
      pop = cells[3].find('span')
      # if there is no value, we're omitting this city
      if(pop is None):
        continue
      pops.append(pop.text.replace(",", ""))
      # get name of cities, in first column
      names.append(cells[0].text.strip())

# generate a dataframe based on the city names & populations
df = pd.DataFrame({"city": names, "population": pops})
df.population = pd.to_numeric(df.population) # cast string to numeric values
df

,city,population
0,Tokyo,37400068
1,Delhi,28514000
2,Shanghai,25582000
3,São Paulo,21650000
4,Mexico City,21581000
...,...,...
76,"Washington, D.C.",5207000
77,Yangon,5157000
78,Alexandria,5086000
79,Jinan,5052000


✏️ *Try to extract another column*

## Examining

You are now able to load and parse data from several formats. At this point, there are plenty of ways to inspect these datasets. We are going to try some simple methods to peek around the datasets. Once you have a tabular dataset ready as a DataFrame, there are quite a few convenient methods to view and explore its contents.

### Head & tail

You could start with looking at the beginning of the dataset with `head()`:

In [14]:
kitas.head()

,name_der_kindertagesbetreuungseinrichtung,stand_vom,barrierefreie_einrichtung,kinderkrippe_0_3_j,tagespflege_0_3_j,padagogisch_begleitete_spielgruppe_0_3_j,kindergarten_3_j_schuleintritt,hort_ab_schuleintritt,andere_kinderbetreuung_ab_3_klasse,strasse,...,abweichende_offnungszeiten,ubernachtung_moglich,schliesstage_von_bis,fruhstuck,mittag,vesper,abendessen,versorgungsart,link_zum_bereich_kinder_und_jugend_der_landeshauptstadt_potsdam,link_zu_anmeldeinformationen_der_einrichtung
0,"Wurzelwerk, AWO, Kita",15.08.2016,Nein,Ja,Nein,Nein,Ja,Nein,Nein,Zum Kahleberg,...,NaN,Nein,NaN,Ja,Ja,Ja,Nein,Mischversorgung,http://www.potsdam.de/kita-tipp,http://www.awo-potsdam.de/awo-bezirksverband/a...
1,"Baumhaus, Kita",07.11.2016,Ja,Nein,Nein,Nein,Ja,Ja,Nein,Sternstr.,...,NaN,Nein,keine,Ja,Ja,Ja,Nein,Mischversorgung,http://www.potsdam.de/kita-tipp,http://www.baumhaus-potsdam.de
2,"Sonnenland, Hort",21.09.2016,Nein,Nein,Nein,Nein,Nein,Ja,Nein,Knobelsdorffstr.,...,Ferien 6 bis 17 Uhr,Nein,NaN,Nein,Ja,Ja,Nein,Eigenversorgung am Standort,http://www.potsdam.de/kita-tipp,https://www.ejf.de/kindertagesstaetten-horte/h...
3,"Pfiffikus, AWO, Kita",08.09.2016,Ja,Ja,Nein,Nein,Ja,Ja,Nein,Pietschkerstr.,...,NaN,Nein,NaN,Ja,Ja,Ja,Nein,Eigenversorgung geliefert,http://www.potsdam.de/kita-tipp,http://www.awo-potsdam.de/einrichtungen-und-di...
4,Evangelische Kindertagesstätte Sonnenblume,01.02.2016,Nein,Ja,Nein,Nein,Ja,Nein,Nein,Bellavitestr.,...,Fr 6 bis 17 Uhr,Nein,1.8.2016-12.8.2016,Ja,Ja,Ja,Nein,Eigenversorgung am Standort,http://www.potsdam.de/kita-tipp,http://www.hoffbauer-bildung.de/kita-sonnenblu...


✏️ *What do you think happens, when you replace `head()` with `tail()` ?*

### Describe & info

You can also ask Pandas to provide some statistical descriptions (which are only applied to the columns containing numeric data):


In [15]:
kitas.describe()

,platze_unbefristet,darunter_betriebskita_unbefristet,platze_befristet,darunter_betriebskita_befristet,befristete_betriebserlaubnis_erlaubnis_gultig_bis_jahr,nur_tagespflege_erste_hilfe_kurs_gultig_bis_jahr
count,136.000000,136.000000,136.000000,135.0,20.000000,1.0
mean,117.727941,1.132353,19.860294,0.0,2016.350000,2015.0
std,81.297302,10.133047,60.033839,0.0,1.225819,NaN
min,0.000000,0.000000,0.000000,0.0,2014.000000,2015.0
25%,54.750000,0.000000,0.000000,0.0,2016.000000,2015.0
50%,105.000000,0.000000,0.000000,0.0,2016.000000,2015.0
75%,161.750000,0.000000,0.000000,0.0,2017.000000,2015.0
max,404.000000,110.000000,269.000000,0.0,2019.000000,2015.0


✏️ *Try any of the other datasets that we parsed above, e.g., `anscombe`*

This may not seem that useful to you yet. You may want to know what kind of datatypes the different columns contain and how many values are present. For this the `info()` method will be of help. Note that strings as displayed as `object`:

In [16]:
kitas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Data columns (total 49 columns):
 #   Column                                                           Non-Null Count  Dtype  
---  ------                                                           --------------  -----  
 0   name_der_kindertagesbetreuungseinrichtung                        137 non-null    object 
 1   stand_vom                                                        137 non-null    object 
 2   barrierefreie_einrichtung                                        137 non-null    object 
 3   kinderkrippe_0_3_j                                               137 non-null    object 
 4   tagespflege_0_3_j                                                137 non-null    object 
 5   padagogisch_begleitete_spielgruppe_0_3_j                         137 non-null    object 
 6   kindergarten_3_j_schuleintritt                                   137 non-null    object 
 7   hort_ab_schuleintritt                       

With this it is now possible to access specific columns by using their names.  But did you notice the long label for the first column? Let's rename the column `name_der_kindertagesbetreuungseinrichtung` into something short and sweet such as: `name`:

In [17]:
kitas = kitas.rename(columns={"name_der_kindertagesbetreuungseinrichtung": "name"})

✏️ *Do you want to rename any other columns?*

### Select & query

We can select an individual column using single [square brackets]:

In [18]:
kitas["name"]

0                                  Wurzelwerk, AWO, Kita
1                                         Baumhaus, Kita
2                                       Sonnenland, Hort
3                                   Pfiffikus, AWO, Kita
4             Evangelische Kindertagesstätte Sonnenblume
                             ...                        
132                                Spielgruppe Waldstadt
133                                   Heilig-Kreuz, Kita
134                                   Grasshoppers, Kita
135    Tagespflegepersonen FidL - Frauen in der Leben...
136         Tagespflegepersonen Treffpunkt Fahrland e.V.
Name: name, Length: 137, dtype: object

… and we can select multiple columns using nested [[square brackets]]:

In [19]:
kitas[["name", "betrieb_e"]]

,name,betrieb_e
0,"Wurzelwerk, AWO, Kita",NaN
1,"Baumhaus, Kita",NaN
2,"Sonnenland, Hort",NaN
3,"Pfiffikus, AWO, Kita",NaN
4,Evangelische Kindertagesstätte Sonnenblume,NaN
...,...,...
132,Spielgruppe Waldstadt,NaN
133,"Heilig-Kreuz, Kita",NaN
134,"Grasshoppers, Kita",NaN
135,Tagespflegepersonen FidL - Frauen in der Leben...,NaN


✏️ *Which columns interest you? Replace `name` and `betrieb_e` with other column labels* 

Note that the `betrieb_e` columns contains a lot of `NaN` - this stands for "Not a Number" and it means here that values are missing.

In order to focus on the rows with available data, we can squeeze in a requirement that we only want those rows, where the values in column `betrieb_e` are not missing, i.e., `notnull()`:

In [20]:
kitas [kitas["betrieb_e"].notnull()] [["name", "betrieb_e"]]

,name,betrieb_e
38,"Kastanienhof, Hort",davon 37 Plätze am Schulstandort ...
76,"Die Buntstifte, Hort",Röhrenstr. 6 /14480 Potsdam / Galileistr. 6 / ...
81,Kindergarten und Hort der Waldorfschule Potsdam,Kita 50 Plätze - Hort 110 Plätze
96,Betriebskita klEinstein,Studentenwerk Potsdam
107,"Geolino, Kita","GFZ, AWI, PIK"


✏️ *Formulate a query on another column:*

It is also possible to directly refer to rows or columns or position. For this there are four related methods for accessing rows, columns, and specific values, either by row and column labels (loc and at) or their integer positions (iloc and iat). Labels can be either indices of rows or names of columns and integer positions refer to the place in the sequence of rows or coolumns: 

- `loc`: access rows and columns by label
- `iloc`: access rows and columns by integer position 
- `at`: access a single value for a row/column label pair
- `iat`: access a single value for a row/column pair by integer position 

To illustrate the differences, we are going set the kita name as the DataFrame's index first:

In [21]:
kitas['name_copy'] = kitas['name']
kitas = kitas.set_index("name_copy")

Now we can get the first entry in the `kitas` DataFrame by integer position… 

In [22]:
kitas.iloc[0]

name                                                                                           Wurzelwerk, AWO, Kita
stand_vom                                                                                                 15.08.2016
barrierefreie_einrichtung                                                                                       Nein
kinderkrippe_0_3_j                                                                                                Ja
tagespflege_0_3_j                                                                                               Nein
padagogisch_begleitete_spielgruppe_0_3_j                                                                        Nein
kindergarten_3_j_schuleintritt                                                                                    Ja
hort_ab_schuleintritt                                                                                           Nein
andere_kinderbetreuung_ab_3_klasse                              

… or by its name:

In [23]:
kitas.loc["Kastanienhof, Hort"]

name                                                                                              Kastanienhof, Hort
stand_vom                                                                                                 01.06.2016
barrierefreie_einrichtung                                                                                         Ja
kinderkrippe_0_3_j                                                                                              Nein
tagespflege_0_3_j                                                                                               Nein
padagogisch_begleitete_spielgruppe_0_3_j                                                                        Nein
kindergarten_3_j_schuleintritt                                                                                  Nein
hort_ab_schuleintritt                                                                                             Ja
andere_kinderbetreuung_ab_3_klasse                              

To retrieve a single value, we need to specify row and column:

In [24]:
kitas.at["Kastanienhof, Hort", "e_mail"]

'kastanienhof-potsdam@froebel-gruppe.de'

In [25]:
kitas.iat[0, 14]

'331.817.186'

Likewise, we can retrieve this value by referring to the respective integer positions of row and column:

Finally, you can also retrieve rows that match a query. With this we are retrieving all the kitas who are operated by AWO:

In [26]:
kitas[kitas['trager'].str.contains('AWO')] 

,name,stand_vom,barrierefreie_einrichtung,kinderkrippe_0_3_j,tagespflege_0_3_j,padagogisch_begleitete_spielgruppe_0_3_j,kindergarten_3_j_schuleintritt,hort_ab_schuleintritt,andere_kinderbetreuung_ab_3_klasse,strasse,...,abweichende_offnungszeiten,ubernachtung_moglich,schliesstage_von_bis,fruhstuck,mittag,vesper,abendessen,versorgungsart,link_zum_bereich_kinder_und_jugend_der_landeshauptstadt_potsdam,link_zu_anmeldeinformationen_der_einrichtung
name_copy,,,,,,,,,,,,,,,,,,,,,
"Wurzelwerk, AWO, Kita","Wurzelwerk, AWO, Kita",15.08.2016,Nein,Ja,Nein,Nein,Ja,Nein,Nein,Zum Kahleberg,...,NaN,Nein,NaN,Ja,Ja,Ja,Nein,Mischversorgung,http://www.potsdam.de/kita-tipp,http://www.awo-potsdam.de/awo-bezirksverband/a...
"Pfiffikus, AWO, Kita","Pfiffikus, AWO, Kita",08.09.2016,Ja,Ja,Nein,Nein,Ja,Ja,Nein,Pietschkerstr.,...,NaN,Nein,NaN,Ja,Ja,Ja,Nein,Eigenversorgung geliefert,http://www.potsdam.de/kita-tipp,http://www.awo-potsdam.de/einrichtungen-und-di...
"Krabbelkäfer, AWO","Krabbelkäfer, AWO",01.01.2016,Nein,Nein,Nein,Ja,Nein,Nein,Nein,Pasteurstr.,...,je nach Angebot,Nein,NaN,Nein,Nein,Nein,Nein,keine Angabe,http://www.potsdam.de/kita-tipp,http://www.awo-potsdam.de/einrichtungen-und-di...
"Turmspatzen, AWO, Kita","Turmspatzen, AWO, Kita",01.09.2016,Ja,Ja,Nein,Nein,Ja,Ja,Nein,Kaiser-Friedrich-Str.,...,NaN,Nein,keine,Ja,Ja,Ja,Nein,Mischversorgung,http://www.potsdam.de/kita-tipp,http://www.awo-potsdam.de/einrichtungen-und-di...
"Wilde Früchtchen, AWO, Kita","Wilde Früchtchen, AWO, Kita",15.12.2015,Nein,Ja,Nein,Nein,Ja,Nein,Nein,Wildbirnenweg,...,NaN,Nein,NaN,Ja,Ja,Ja,Nein,Eigenversorgung am Standort,http://www.potsdam.de/kita-tipp,http://www.awo-potsdam.de/einrichtungen-und-di...
"Eltern-Kind-Gruppe im EKIZ, AWO","Eltern-Kind-Gruppe im EKIZ, AWO",01.01.2016,Nein,Nein,Nein,Ja,Nein,Nein,Nein,Röhrenstr.,...,je nach Angebot,Nein,NaN,Nein,Nein,Nein,Nein,keine Angabe,http://www.potsdam.de/kita-tipp,http://www.awo-potsdam.de/einrichtungen-und-di...
"Am Babelsberg, AWO, Kita","Am Babelsberg, AWO, Kita",31.03.2016,Nein,Ja,Nein,Nein,Ja,Ja,Nein,Grenzstr.,...,NaN,Nein,keine Sommerschließzeiten,Ja,Ja,Ja,Nein,Eigenversorgung geliefert,http://www.potsdam.de/kita-tipp,http://www.awo-potsdam.de/einrichtungen-und-di...
"Nuthegeister, AWO, Hort für hör-, sprach-, lern-, und geistig behinderte Kinder","Nuthegeister, AWO, Hort für hör-, sprach-, ler...",22.04.2016,Ja,Nein,Nein,Nein,Ja,Nein,Nein,Bisamkiez,...,Ferien:8-17 Uhr und nach Bedarf ab 6 Uhr,Nein,keine,Nein,Ja,Ja,Nein,Fremdversorgung,http://www.potsdam.de/kita-tipp,http://www.awo-potsdam.de/einrichtungen-und-di...
"Integrationskita Sternschnuppe, AWO","Integrationskita Sternschnuppe, AWO",01.11.2015,Nein,Ja,Nein,Nein,Ja,Ja,Nein,Max-Born-Str.,...,NaN,Nein,NaN,Ja,Ja,Ja,Nein,Eigenversorgung am Standort,http://www.potsdam.de/kita-tipp,http://www.awo-potsdam.de/einrichtungen-und-di...


✏️ *How about all the entries that have "Hort" in their name?*

… or those who have sparrows in their name. Because ‘Spatzen’ is occurring both by itself and as part of compound words, we are querying for uppercase and lowercase versions using the IGNORECASE flag of the regular expression module `re`:

In [27]:
import re # regular expressions module, use for a case-insensitive query

kitas[kitas['name'].str.contains('Spatzen', flags=re.IGNORECASE, regex=True)] ["name"]

name_copy
Turmspatzen, AWO, Kita                                                                Turmspatzen, AWO, Kita
Spatzennest, Kita                                                                          Spatzennest, Kita
Evangelische integrative Kindertagesstätte Nuthespatzen    Evangelische integrative Kindertagesstätte Nut...
Spatzenhaus, AWO, Kita                                                                Spatzenhaus, AWO, Kita
Weberspatzen, AWO, Hort                                                              Weberspatzen, AWO, Hort
Name: name, dtype: object

✏️ *Now it's time for you to wrangle data! If you have not done so yet, follow all the pencils and change things around!*

## Sources
- [Pandas Tutorial: DataFrames in Python - DataCamp](https://www.datacamp.com/community/tutorials/pandas-tutorial-dataframe-python)
- [The ElementTree XML API](https://docs.python.org/3/library/xml.etree.elementtree.html)
- [Where do Mayors Come From? Querying Wikidata with Python and SPARQL - Towards Data Science](https://towardsdatascience.com/where-do-mayors-come-from-querying-wikidata-with-python-and-sparql-91f3c0af22e2)
- [External data: Local Files, Drive, Sheets, and Cloud Storage - Colaboratory](https://colab.research.google.com/notebooks/io.ipynb)
- [Examining Data Using Pandas | Linux Journal](https://www.linuxjournal.com/content/examining-data-using-pandas)
- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Regular Expressions in Python](https://docs.python.org/3/library/re.html)